<a href="https://colab.research.google.com/github/mahdieslaminet/ML-Final-Project/blob/main/Futsal_Suitability_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ===============================
# 1️⃣ Import libraries
# ===============================
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ===============================
# 2️⃣ Load dataset
# ===============================
url = "https://raw.githubusercontent.com/Zahra-Alikhani2004/ML_Final-project-/main/Iran_Futsal_Dataset/Iran_Futsal_Players.csv"
df = pd.read_csv(url)

# ===============================
# 3️⃣ Convert Persian numbers
# ===============================
def persian_to_english_num(s):
    if pd.isnull(s):
        return np.nan
    s = str(s)
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    english_digits = "0123456789"
    for p, e in zip(persian_digits, english_digits):
        s = s.replace(p, e)
    s = s.replace('٫', '.')
    try:
        return float(s)
    except:
        return np.nan

numeric_cols = ['سن تقریبی','قد (متر)','وزن تقریبی (کیلو)','سن شروع حرفه‌ای']
for col in numeric_cols:
    df[col] = df[col].apply(persian_to_english_num)

# ===============================
# 4️⃣ Rename columns
# ===============================
df.rename(columns={
    'سن تقریبی':'Age',
    'قد (متر)':'Height',
    'وزن تقریبی (کیلو)':'Weight',
    'سن شروع حرفه‌ای':'Start_Age'
}, inplace=True)

# ===============================
# 5️⃣ Create target (Suitable or Not)
# ===============================
df['Suitable_For_Futsal'] = (
    (df['Height'].between(1.65, 1.85)) &
    (df['Weight'].between(60, 85)) &
    (df['Start_Age'] <= 23) &
    (df['Age'] <= 45)
).astype(int)

# ===============================
# 6️⃣ Features & Target
# ===============================
X = df[['Age','Height','Weight','Start_Age']]
y = df['Suitable_For_Futsal']

# حذف سطرهای ناقص
data = pd.concat([X, y], axis=1).dropna()
X = data[['Age','Height','Weight','Start_Age']]
y = data['Suitable_For_Futsal']

# ===============================
# 7️⃣ Train-test split (NO stratify error)
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

# ===============================
# 8️⃣ Train model
# ===============================
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)
model.fit(X_train, y_train)

# ===============================
# 9️⃣ Evaluation
# ===============================
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, target_names=["Not Suitable","Suitable"]))

Accuracy: 1.0

Classification Report:

              precision    recall  f1-score   support

Not Suitable       1.00      1.00      1.00         1
    Suitable       1.00      1.00      1.00         5

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

